In [1]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, layers
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from PIL import Image
import numpy as np

##驗證是否安裝tensorflow
import tensorflow as tf
print(tf.__version__)
from PIL import Image
print(Image.__version__)

## 資料目錄來源 以資料夾當作分類 , e.g. Datasets/cats, Datasets/dogs, ...
src_dir = r'Datasets/'
## 單次預測圖片
predict_img = r'Datasets/test1.png'

## 影像讀取處理
datagen = ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input)
train_generator = datagen.flow_from_directory(src_dir, target_size=(224, 224), batch_size=20, subset='training')
valid_generator = datagen.flow_from_directory(src_dir, target_size=(224, 224), batch_size=20, subset='validation')

# 初始化 MobileNetV2，明確指定輸入尺寸
mobilenetV2 = MobileNetV2(include_top=False, pooling='avg', input_shape=(224, 224, 3))

## 模型建立
mobilenetV2 = MobileNetV2(include_top=False, pooling='avg')
for mlayer in mobilenetV2.layers:
    mlayer.trainable = False
mobilenetV2output = mobilenetV2.layers[-1].output
fc = layers.Dense(units=train_generator.num_classes, activation='softmax', name='custom_fc') (mobilenetV2output)
classification_model = Model(
            inputs=mobilenetV2.inputs,
            outputs=fc)
classification_model.compile(loss='categorical_crossentropy', optimizer='Adam')

## 模型訓練
history = classification_model.fit(train_generator, epochs=10, validation_data=valid_generator)

## 單次預測
true_labels_dict = {}
for key in train_generator.class_indices:
    true_labels_dict[train_generator.class_indices[key]] = key

def pred(img_path):
    img = preprocess_input(np.array(Image.open(img_path).convert('RGB')))
    img = np.array([img])
    result_prob = classification_model.predict(img).tolist()[0]
    max_index = result_prob.index(max(result_prob))
    print(true_labels_dict[max_index], result_prob[max_index])

pred(predict_img)

2.16.1
10.3.0
Found 6404 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.


C:\Users\owner\AppData\Local\Temp\ipykernel_11368\2884025764.py:27: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenetV2 = MobileNetV2(include_top=False, pooling='avg')


Epoch 1/10


C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


321/321 ━━━━━━━━━━━━━━━━━━━━ 84s 246ms/step - loss: 0.1310 - val_loss: 0.0470
Epoch 2/10
321/321 ━━━━━━━━━━━━━━━━━━━━ 76s 237ms/step - loss: 0.0364 - val_loss: 0.0410
Epoch 3/10
321/321 ━━━━━━━━━━━━━━━━━━━━ 77s 241ms/step - loss: 0.0271 - val_loss: 0.0432
Epoch 4/10
321/321 ━━━━━━━━━━━━━━━━━━━━ 79s 247ms/step - loss: 0.0228 - val_loss: 0.0453
Epoch 5/10
321/321 ━━━━━━━━━━━━━━━━━━━━ 76s 237ms/step - loss: 0.0187 - val_loss: 0.0645
Epoch 6/10
321/321 ━━━━━━━━━━━━━━━━━━━━ 76s 237ms/step - loss: 0.0150 - val_loss: 0.0529
Epoch 7/10
321/321 ━━━━━━━━━━━━━━━━━━━━ 77s 239ms/step - loss: 0.0111 - val_loss: 0.0498
Epoch 8/10
321/321 ━━━━━━━━━━━━━━━━━━━━ 76s 236ms/step - loss: 0.0076 - val_loss: 0.0538
Epoch 9/10
321/321 ━━━━━━━━━━━━━━━━━━━━ 76s 236ms/step - loss: 0.0089 - val_loss: 0.0524
Epoch 10/10
321/321 ━━━━━━━━━━━━━━━━━━━━ 76s 238ms/step - loss: 0.0072 - val_loss: 0.0526
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
cats 0.9999536275863647
